In [1]:
import numpy as np
import pandas as pd
import sympy
from sympy import *
from operator import mul
from functools import reduce
import mpmath as mp
from mpmath import fac
import functools
from sympy import init_printing
from sympy.core.decorators import _sympifyit, call_highest_priority
from IPython.display import display_latex
init_printing()

In [33]:
class LieOperator:
    '''
    Defined algebra for the Lie operator:
    Addition: Hamiltonian functions get added into a new Lie operator
    Multiplication: Poisson bracket gets applied
    CAUTION: NOT ASSOCIATIVE

    INPUT: Hamiltonian, generalized coordinates and momenta
    '''
    #Initialize the operator
    def __init__(self, ham, indep_coords, indep_mom):
        self._ham = ham 
        if not isinstance(indep_coords, list): #create list of coords
            indep_coords = [indep_coords]
        if not isinstance(indep_mom, list): #create list of momenta
            indep_mom = [indep_mom]

        #same number of coordinates and momenta have to be entered
        assert(len(indep_coords)==len(indep_mom)),'The number of coords and momenta is not the same'
        self._indep_coords = indep_coords
        self._indep_mom = indep_mom
    
    @property
    def ham(self):
        return self._ham
    
    @ham.setter
    def ham(self, other):
        self._ham = self._ham.subs(self.ham, other).doit()
        
    @property
    def indep_coords(self):
        return self._indep_coords
    
    @indep_coords.setter
    def indep_coords(self, other):
        self._indep_coords = other
        
    @property
    def indep_mom(self):
        return self._indep_mom
    
    @indep_mom.setter
    def indep_mom(self, other):
        self._indep_mom = other
    
    #Transform all variable to Lie Operators
    def MakeLieOperator(op):
        def CheckLieOperator(self,other):
            if not isinstance(other, LieOperator):
                other = LieOperator(other, self.indep_coords,self.indep_mom)
                return op(self,other)
        return CheckLieOperator
    
    #Algebraic properties 

    #Define Addition
    @MakeLieOperator
    def __add__(self,other):
        return LieOperator(self.ham + other.ham, self.indep_coords,self.indep_mom)
    
    @MakeLieOperator
    def __radd__(self,other):
        return LieOperator(other.ham + self.ham,self.indep_coords,self.indep_mom)
        
        
    #Define Multiplication as Poisson Bracket
    def __mul__(self, other):
        return self.Poisson(other)
    
    @MakeLieOperator
    def __rmul__(self,other):
        return other.Poisson(self)
    
    #Calcualte Poisson bracket with input function
    def Poisson(self,other):
        h = 0
        for qi,pi in zip(self.indep_coords,self.indep_mom):
            h += self.ham.diff(qi) * other.ham.diff(pi) 
            h -= self.ham.diff(pi) * other.ham.diff(qi)

        return h

In [25]:
class LieMap(LieOperator):
    '''
    Defined algebra for the Lie Map:
    Inherits the structure of the Lie Operator
    Addition: Lie maps get added into a new Lie map
    Multiplication: Baker-Cambell-Hausdorff Formula is used 
    CAUTION: NOT ASSOCIATIVE

    INPUT: Hamiltonian, generalized coordinates and momenta, ExpPower, BCHPower
    ExpPower: cutoff for the exponential function
    BCHPower: cutoff for the Baker-Cambell-Hausdorff series
    
    For operations the higher power of the two elements will be selected
    '''
    
    def __init__(self, ham, indep_coords, indep_mom, ExpPower=6,BCHPower=2):
        LieOperator.__init__(self, ham, indep_coords, indep_mom)
        self.ExpN = ExpPower
        self.BCHN = BCHPower
        
    #Define Multiplication
    '''
    Multiplication:
    1) Lie Map * Lie Map :  Application of the BCH series with cutoff
    2) Lie Map * Variable : Application of the exponential function cutoff
        to the variable
    '''
    def __mul__(self, other):
        if isinstance(other, LieMap):
            return LieMap(BCH(self.ham,other.ham,max(self.BCHN,other.BCHN)),
                          self.indep_coords,
                          self.indep_mom,
                          max(self.ExpN,other.ExpN),
                          max(self.BCHN,other.BCHN))
        else:
            return self.exp(self,other,self.ExpN)
    

    def __rmul__(self,other):
        if not isinstance(other, LieMap):
            print('Illegal operation: Right operator is not a Lie Map.')
            return
        return LieMap(BCH(other.ham,self.ham,max(self.BCHN,other.BCHN)),
                          self.indep_coords,
                          self.indep_mom,
                          max(self.ExpN,other.ExpN),
                          max(self.BCHN,other.BCHN))
    
    

In [26]:
x, p = sympy.symbols('x p')
H = sympy.Function('H', commutative=False)(x,p)
g = sympy.Function('g', commutative=False)(x,p)
la = LieOperator(H,x,p)
lb = LieOperator(g,x,p)

In [27]:
ma = LieMap(H,x,p,5,5)
mb = LieMap(g,x,p,2,3)

In [34]:
n = 4
def pn(n):
    F = sympy.Matrix(np.zeros((n+1,n+1)))
    for i in range(n+1):
        for j in range(n+1):
            if (j-i)>=0:
                F[i,j] = sympy.Rational(1,int(fac(j-i))) # keep it rational !!!!


    
    G = sympy.Matrix(np.zeros((n+1,n+1)))
    for i in range(n+1):
        for j in range(n+1):
            if (j-i)>=0:
                if i==j:
                    sigprod = 1
                else:
                    l = sympy.symbols('s{}:{}'.format(i,j))
                    sigprod = reduce(sympy.Mul,l)
                G[i,j] = sympy.Mul(sympy.Rational(1,int(np.math.factorial(j-i))),sigprod, evaluate=False)



    qthpower = sympy.Matrix(np.eye(n+1))

    FGm1 = sympy.nsimplify(F*G - qthpower)

    FGm1


    summ = 0

    for q in range(1,n+1):
        qthpower = sympy.nsimplify(sympy.MatMul(qthpower, FGm1, evaluate=False))

        summ += sympy.nsimplify((-1)**(q-1)/q * qthpower)[0,n] # q-1 because mathematica starts at 1 and not 0, there for sign change

    return sympy.expand(summ)

def translated(op1,op2,n):
    l = sympy.symbols('s{}:{}'.format(0,n))
    temp = reduce(functools.partial(sympy.Mul,evaluate=False),[k*k for k in l])
    temp = sympy.expand(sympy.Mul(temp, pn(n), evaluate=False),evaluate=False)
    with sympy.evaluate(False):
        ln = sympy.symbols('r{}:{}'.format(0,n), commutative=False)
        for i, s in enumerate(l):
            temp = temp.subs(l[i],ln[i])
        for i in range(n):
            f = sympy.Symbol('f',commutative=False)
            g = sympy.Symbol('g',commutative=False)
            temp = temp.subs(ln[i]**3,g)
            temp = temp.subs(ln[i]**2,f)
            temp = temp.subs(g**0,1)
        
        temp = temp.subs(f,op1)
        temp = temp.subs(g,op2)
    
    return temp

def HCB(op1,op2,n):
    s = 0
    for i in range(1,n+1):
        s+= translated(op1,op2,i)
    return s

In [39]:
translated(la.ham,lb.ham,3)